In [1]:
import pandas as pd
import numpy as np
import collections
from  more_itertools import unique_everseen
from collections import Counter

from itertools import chain
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

from sklearn.metrics import f1_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score

import pickle

from training import *

In [2]:
# Загрузка данных
df = pd.read_csv('train_classif_norm_check.txt', sep='\t',encoding='cp1251', engine='python', header=0, error_bad_lines=False)
df_cat = pd.read_csv('train_check_no.txt', sep='\t',encoding='cp1251', engine='python', header=0, error_bad_lines=False)
df_cat.head()

,текст,бин тэги,тэги,текст анамнеза,is_med,is_food,is_env,Unnamed: 7
0,"В году возобновление стенокардии при ФН, стаци...",A,B,в год возобновление стенокардия при фн стацион...,1,0,0,NaN
1,С г нарастание явлений ХСН до III ФК. Неоднокр...,A,B,с год нарастание явление хсн до iii фк неоднок...,1,0,0,NaN
2,г. прогрессирование клиники стенокардии до III...,A,B,год прогрессирование клиника стенокардия до ii...,1,0,0,NaN
3,Состояние после оперативного лечения м. берцов...,A,B,состояние после оперативный лечение метр берцо...,1,0,0,NaN
4,"Начата аб терпия левофлоксацином, после перво...",A,B,в год возобновление стенокардия при фн стацион...,1,0,0,NaN


In [3]:
df.head()

,текст,бин тэги,пред бин тэги,тэги,текст анамнеза
0,"В году возобновление стенокардии при ФН, стаци...",A,A,B,в год возобновление стенокардия при фн стацион...
1,"ЛЖ, стац. лечение в ФЦ, тромболизис. ангинозны...",C,C,D,лж стац лечение в фц тромболизис ангинозный бо...
2,РТСА со стентированием ПМЖА C без лекарственно...,C,C,D,рт са с стентирование пмж c без лекарственный ...
3,"С. г. переносит острый бронхит, проводилась ан...",C,C,D,с год переносить острый бронхит проводиться ан...
4,Стац. лечение по мж с представлением о повторн...,C,C,C,стац лечение по мж с представление о повторный...


In [4]:
def f(a,b,c):
    return a+b+c

df_cat['num'] = df_cat.apply(lambda x: f(x.is_med, x.is_food, x.is_env), axis=1)

df_cat_new = df_cat.loc[df_cat['num'] != 0]
print('Количество категорий аллергии на запись:\n',df_cat_new['num'].value_counts())

df_cat_new['is_food'] = df_cat_new['is_food'].replace(1, 'A').replace(0, 'B')
df_cat_new['is_env'] = df_cat_new['is_env'].replace(1, 'A').replace(0, 'B')
df_cat_new['is_med'] = df_cat_new['is_med'].replace(1, 'A').replace(0, 'B')

Количество категорий аллергии на запись:
 1    7741
2    1306
3      93
Name: num, dtype: int64


## Фильтрация аллергоанамнезов

In [5]:
#Подготовка данных
train, val = train_test_split(df, test_size=0.33, random_state=42)
X_train, y_train_cv = train['текст анамнеза'].values, train['бин тэги'].values
X_val, y_val_cv = val['текст анамнеза'].values, val['бин тэги'].values

#Создание словаря
DICT_SIZE, WORDS_TO_INDEX, INDEX_TO_WORDS = make_dict(X_train, y_train_cv)

from scipy import sparse as sp_sparse

#Мешок слов
X_train_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_train])
X_val_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_val])

#Подготовка и преобразование таргетов
from sklearn.preprocessing import MultiLabelBinarizer
tags_counts = Counter(chain.from_iterable([i for i in y_train_cv]))
mlb = MultiLabelBinarizer(classes=sorted(tags_counts.keys()))
y_train = mlb.fit_transform(y_train_cv)
y_val = mlb.fit_transform(y_val_cv)

Число уникальных слов 7272


In [6]:
model_lr = train_log_regression(X_train_mybag, y_train_cv)
pred_lr = mlb.fit_transform(model_lr.predict(X_val_mybag))
scores(y_val, pred_lr)

F-score: 0.9431325301204819
Precision: 0.9179327043112209
Recall: 0.9073737096700849


In [7]:
top_AL_p, top_AL_n  = top_words(model_lr, 'A', mlb.classes, INDEX_TO_WORDS, WORDS_TO_INDEX.keys(),'lr')

Tag:	A
Top positive words:	аллергоанамнез, пенициллин, престариум, анальгин, диротон, нитропрепарат, связь, аллергодерматит, ринит, аллергический, зуд, отечь, кашель, сыпь, аллергия, крапивница, плохой, непереносимость, аллергодерматить, нитрат
Top negative words:	диувера, переноситься, подозрение, хорошо, нагрузка, быть, отмечать, особенность, удовлетворительный, перенос, физ, гипоаллергенный, субъективно, рча, фн, неаллергический, спокойный, отрица, нет, отрицать



In [8]:
filename = 'allerg_classifier.sav'
pickle.dump(model_lr, open(filename, 'wb'))

## Пищевая аллергия

In [9]:
#Подготовка данных
train, val = train_test_split(df_cat_new, test_size=0.33, random_state=42)
X_train, y_train_cv = train['текст анамнеза'].values, train['is_food'].values
X_val, y_val_cv = val['текст анамнеза'].values, val['is_food'].values

#Создание словаря
DICT_SIZE, WORDS_TO_INDEX, INDEX_TO_WORDS = make_dict(X_train, y_train_cv)

from scipy import sparse as sp_sparse

#Мешок слов
X_train_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_train])
X_val_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_val])

#Подготовка и преобразование таргетов
from sklearn.preprocessing import MultiLabelBinarizer
tags_counts = Counter(chain.from_iterable([i for i in y_train_cv]))
mlb = MultiLabelBinarizer(classes=sorted(tags_counts.keys()))
y_train = mlb.fit_transform(y_train_cv)
y_val = mlb.fit_transform(y_val_cv)


Число уникальных слов 5340


In [10]:
model_lr = train_log_regression(X_train_mybag, y_train_cv)
pred_lr = mlb.fit_transform(model_lr.predict(X_val_mybag))
scores(y_val, pred_lr)

F-score: 0.9532648326151807
Precision: 0.9320814247932581
Recall: 0.875682938746502


In [11]:
top_AL_p, top_AL_n  = top_words(model_lr, 'A', mlb.classes, INDEX_TO_WORDS, WORDS_TO_INDEX.keys(),'lr')

Tag:	A
Top positive words:	мандарин, солодка, сорбифер, спирт, стоматология, орех, комаров, яйцо, алкоголь, красный, рыба, мед, стоматит, молоко, продукт, цитрусовый, лактоза, шоколад, пищевой, клубника
Top negative words:	крапивница, гормон, аспирин, фолиевый, новокаин, смешанный, субкомпенсировать, лек, самостоятельно, пластырь, переносить, тироксин, пенициллин, отмечаться, пятно, лекарство, кура, эритромицин, поливалентный, сок



In [12]:
filename = 'food_classifier.sav'
pickle.dump(model_lr, open(filename, 'wb'))

## Средовая аллергия

In [13]:
#Подготовка данных
train, val = train_test_split(df_cat_new, test_size=0.33, random_state=42)
X_train, y_train_cv = train['текст анамнеза'].values, train['is_env'].values
X_val, y_val_cv = val['текст анамнеза'].values, val['is_env'].values

#Создание словаря
DICT_SIZE, WORDS_TO_INDEX, INDEX_TO_WORDS = make_dict(X_train, y_train_cv)

from scipy import sparse as sp_sparse

#Мешок слов
X_train_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_train])
X_val_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_val])

#Подготовка и преобразование таргетов
from sklearn.preprocessing import MultiLabelBinarizer
tags_counts = Counter(chain.from_iterable([i for i in y_train_cv]))
mlb = MultiLabelBinarizer(classes=sorted(tags_counts.keys()))
y_train = mlb.fit_transform(y_train_cv)
y_val = mlb.fit_transform(y_val_cv)


Число уникальных слов 5340


In [14]:
model_lr = train_log_regression(X_train_mybag, y_train_cv)
pred_lr = mlb.fit_transform(model_lr.predict(X_val_mybag))
scores(y_val, pred_lr)

F-score: 0.9320517069937023
Precision: 0.9026945310130232
Recall: 0.8773299538605661


In [15]:
top_AL_p, top_AL_n  = top_words(model_lr, 'A', mlb.classes, INDEX_TO_WORDS, WORDS_TO_INDEX.keys(),'lr')

Tag:	A
Top positive words:	сульфаниламидный, краситель, никотиновый, краска, магнезия, солнце, фотодерматоз, холод, укус, металл, поллиноз, пыльца, пыль, лейкопластырь, холодовый, цветение, шерсть, пластырь, сульфаниламид, бытовой
Top negative words:	интоксикация, тонзиллэктомия, ацетилсалициловый, поливитамин, милдронат, переносить, фолиевый, тримекаин, ангионевротич, сторона, анафилактический, дигоксина, баралгин, сорный, канефрон, пятно, гептрал, книжный, красносодержащий, лекарство



In [16]:
filename = 'env_classifier.sav'
pickle.dump(model_lr, open(filename, 'wb'))

## Медикаментозная аллергия

In [17]:
#Подготовка данных
train, val = train_test_split(df_cat_new, test_size=0.33, random_state=42)
X_train, y_train_cv = train['текст анамнеза'].values, train['is_med'].values
X_val, y_val_cv = val['текст анамнеза'].values, val['is_med'].values

#Создание словаря
DICT_SIZE, WORDS_TO_INDEX, INDEX_TO_WORDS = make_dict(X_train, y_train_cv)

from scipy import sparse as sp_sparse

#Мешок слов
X_train_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_train])
X_val_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_val])

#Подготовка и преобразование таргетов
from sklearn.preprocessing import MultiLabelBinarizer
tags_counts = Counter(chain.from_iterable([i for i in y_train_cv]))
mlb = MultiLabelBinarizer(classes=sorted(tags_counts.keys()))
y_train = mlb.fit_transform(y_train_cv)
y_val = mlb.fit_transform(y_val_cv)


Число уникальных слов 5340


In [18]:
model_lr = train_log_regression(X_train_mybag, y_train_cv)
pred_lr = mlb.fit_transform(model_lr.predict(X_val_mybag))
scores(y_val, pred_lr)

F-score: 0.9615512098110706
Precision: 0.9438051241835982
Recall: 0.9343467717688276


In [19]:
top_AL_p, top_AL_n  = top_words(model_lr, 'A', mlb.classes, INDEX_TO_WORDS, WORDS_TO_INDEX.keys(),'lr')

Tag:	A
Top positive words:	аба, ампициллин, эуфиллина, бисептол, йодомарин, полиаллергия, димедрол, аспирин, пластырь, переносить, анальгин, лекарство, препарат, йод, бициллина, антибиотик, поливалентный, пенициллин, новокаин, медикамент
Top negative words:	дом, растение, шоколад, краска, зверобой, шерсть, поллиноз, глютен, аллерген, пищевой, клубника, металл, солнце, пыльца, пыль, укус, холодовый, фотодерматоз, холод, лактоза



In [20]:
filename = 'med_classifier.sav'
pickle.dump(model_lr, open(filename, 'wb'))

## Фильтрация реакций

In [ ]:
df_react = df.loc[df['бин тэги']=='A']

#Подготовка данных
train, val = train_test_split(df_react, test_size=0.33, random_state=42)
X_train, y_train_cv = train['текст анамнеза'].values, train['тэги'].values
X_val, y_val_cv = val['текст анамнеза'].values, val['тэги'].values

#Создание словаря
DICT_SIZE, WORDS_TO_INDEX, INDEX_TO_WORDS = make_dict(X_train, y_train_cv)

from scipy import sparse as sp_sparse

#Мешок слов
X_train_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_train])
X_val_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_val])

#Подготовка и преобразование таргетов
from sklearn.preprocessing import MultiLabelBinarizer
tags_counts = Counter(chain.from_iterable([i for i in y_train_cv]))
mlb = MultiLabelBinarizer(classes=sorted(tags_counts.keys()))
y_train = mlb.fit_transform(y_train_cv)
y_val = mlb.fit_transform(y_val_cv)

Число уникальных слов 5697


In [ ]:
model_lr = train_log_regression(X_train_mybag, y_train_cv)
pred_lr = mlb.fit_transform(model_lr.predict(X_val_mybag))
scores(y_val, pred_lr)

In [ ]:
top_AL_p, top_AL_n  = top_words(model_lr, 'A', mlb.classes, INDEX_TO_WORDS, WORDS_TO_INDEX.keys(),'lr')

In [ ]:
filename = 'react_classifier.sav'
pickle.dump(model_lr, open(filename, 'wb'))